In [224]:
import math
import warnings
from datetime import datetime
start=datetime.now()
from IPython.display import display
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import linear_model
import statsmodels.formula.api as smf
from sklearn.model_selection import cross_val_score
from sklearn import neighbors
from sklearn import tree,model_selection
from sklearn.grid_search import GridSearchCV
# A convenience for displaying visualizations.
from IPython.display import Image
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.ensemble import RandomForestRegressor
# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format


In [225]:
# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
start=datetime.now()
rfc = ensemble.RandomForestRegressor()


We are creating dataframe of the the data in .csv format and are renaming the column name

In [226]:

data = pd.read_csv('C:/Personal/09142640/Downloads/Car_sales_U.csv')
data.columns.values[3]='Year_Resale_Value'
data_req=data.loc[:,["Sales_in_thousands","Year_Resale_Value","Price_in_thousands","Engine_size","Horsepower","Wheelbase","Width","Length","Curb_weight","Fuel_capacity","Fuel_efficiency","Power_perf_factor"]]
data_na=data_req.fillna(0)


Now, we are taking X and Y variable with X as input variable with the columns except 1 column which is for the output variable

In [227]:
Y = data_na[['Sales_in_thousands']].astype('int')
X = data_na.drop(columns='Sales_in_thousands').astype('int')
X.head(5)


,Year_Resale_Value,Price_in_thousands,Engine_size,Horsepower,Wheelbase,Width,Length,Curb_weight,Fuel_capacity,Fuel_efficiency,Power_perf_factor
0,16,21,1,140,101,67,172,2,13,28,58
1,19,28,3,225,108,70,192,3,17,25,91
2,18,0,3,225,106,70,192,3,17,26,0
3,29,42,3,210,114,71,196,3,18,22,91
4,22,23,1,150,102,68,178,2,16,27,62


Now, we will use Grid searchcv ,Holdout,Cross Validation to validate the accuracy of random forest model

In [228]:
# gridsearchCV

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20)
rfc = ensemble.RandomForestRegressor(max_depth=10,n_estimators= 100)
dt_grid={'max_depth':[3,4,5],'max_features': [11]}
grid_class=model_selection.GridSearchCV(rfc,dt_grid,cv=6)
grid_class.fit(X_train, y_train)


C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anacond

GridSearchCV(cv=6, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [3, 4, 5], 'max_features': [11]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [229]:
#Displaying the parameters using Grid searchcv
results = grid_class.cv_results_
print(results.get('params'))
print(results.get('mean_test_score'))
print(results.get('mean_train_score'))
print(grid_class.best_params_)
print(grid_class.best_score_)
final_model = grid_class.best_estimator_
print(final_model)


[{'max_depth': 3, 'max_features': 11}, {'max_depth': 4, 'max_features': 11}, {'max_depth': 5, 'max_features': 11}]
[0.06017809 0.01072284 0.04279258]
[0.5689302  0.6549653  0.73087399]
{'max_depth': 3, 'max_features': 11}
0.06017809184873869
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=3,
           max_features=11, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)


C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


In [230]:
rfc.fit(X_train, y_train)
print('Training score with 20% Holdout: ' + str(rfc.score(X_train, y_train)))
print('Testing score with 20% Holdout: ' + str(rfc.score(X_test, y_test)))

Training score with 20% Holdout: 0.8694554193051707
Testing score with 20% Holdout: -0.09906141001815105


C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


After using the holdout  groups, it looks like the training data is having good accuracy but the testing data is overfitting the model .We played around with max_depth=10,n_estimators= 100 and tried to change the value to get good accuracy and reduced overfitting values.Now, we will check the accuracy of the model with cross validation with folds=10.It should be wither 5 or 10, thats one of the better practices

In [231]:
#Cross Validation
from sklearn.model_selection import cross_val_score
print ("Training Data accuracy: ",cross_val_score(rfc, X_train,y_train, cv=5))
print ("Testing Data accuracy: ",cross_val_score(rfc, X_test,y_test, cv=5))

C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anacond

Training Data accuracy:  [ 0.35182503  0.18577322  0.01453529 -0.79056632 -0.30254571]


C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anacond

Testing Data accuracy:  [-0.52856276 -1.56142974 -0.23482976 -0.95732565 -3.39623542]


C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


In [ ]:
After doing the cross validation,we are seeing overfitting in the dataset.

In [232]:
#Displaying the important feature scores

#decision_tree.fit(X_train, y_train)
rfc.fit(X_train, y_train)
print(rfc.feature_importances_)


[0.11469697 0.12561468 0.03837146 0.07543955 0.17916136 0.10283994
 0.17399281 0.01268269 0.0553635  0.03390512 0.08793194]


C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


Now, we will try to get the best number of features, here we have observed that maximum percentage is with 8 features so we will take these 8 features

In [233]:
# And let's see what percentage of decisions are accounted for with the top X features.
importances = rfc.feature_importances_
indices = np.argsort(importances)[::-1]
top_indices = indices[:55]
for n_idx in range(2, 14, 2):
    print('Percentage of decisions accounted for with {} features: {}'.format(n_idx, importances[indices[:n_idx]].sum()))
print('Total Features: ' + str(len(indices)))

Percentage of decisions accounted for with 2 features: 0.35315416211499817
Percentage of decisions accounted for with 4 features: 0.5934658101755149
Percentage of decisions accounted for with 6 features: 0.7842376860619443
Percentage of decisions accounted for with 8 features: 0.9150407269864783
Percentage of decisions accounted for with 10 features: 0.9873173083643632
Percentage of decisions accounted for with 12 features: 0.9999999999999997
Total Features: 11


Let's reduce to 8 features since we could potentially account for 93% of the decisions from the Random Forest.

In [234]:
#Using PCA
n_indices = 8
X_train.columns[indices[:n_indices]]


Index(['Wheelbase', 'Length', 'Price_in_thousands', 'Year_Resale_Value',
       'Width', 'Power_perf_factor', 'Horsepower', 'Fuel_capacity'],
      dtype='object')

In [235]:
X_train_rfc = X_train.loc[:, X_train.columns[indices[:n_indices]]]
X_test_rfc = X_test.loc[:, X_train.columns[indices[:n_indices]]]
print(X_train_rfc.shape, X_test_rfc.shape)

(124, 8) (32, 8)


In [236]:
X_pca = PCA(n_components=8)
X_pca.fit(X_train)
X_train_pca = X_pca.transform(X_train)
X_test_pca = X_pca.transform(X_test)


By using PCA, we can see 8 features are closely correlated to the output variable so we will keep them
Now we will use these models after doing PCA-

1.GridsearchCV

2.R-squared

3.Mean Square Error


In [237]:
#Using Grid search  for finding the best set of parameters with random Forest model
dt_grid_pca={'max_depth':[10],'max_features': [8]}
grid_DT_pca=model_selection.GridSearchCV(rfc,dt_grid_pca,cv=6)
grid_DT_pca.fit(X_train, y_train)

#grid_class_pca=model_selection.GridSearchCV(random_f_pca,dt_grid_pca,refit=True,return_train_score=True)
#grid_class_pca.fit(X_train_pca, y_train_pca)
#Displaying the parameters using Grid search
results_pca = grid_DT_pca.cv_results_
print(results_pca.get('params'))
print(results_pca.get('mean_test_score'))
print(results_pca.get('mean_train_score'))
print(grid_class_pca.best_params_)
print(grid_class_pca.best_score_)
final_model_pca = grid_DT_pca.best_estimator_
print(final_model_pca)


C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anacond

[{'max_depth': 10, 'max_features': 8}]
[-0.03784745]
[0.85985436]
{'criterion': 'mse', 'max_depth': 4}
-0.23421129935445287
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features=8, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)


C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:740: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


In [238]:
# Calculating model accuracy using R square
print("R-squared :" ,grid_DT_pca.score(X_train,y_train))


R-squared : 0.8646145256112294


It looks like the R-squared got 86.4% accuracy.Now we will determine mean square error

In [239]:
#Validating model accuracy using mean square error

y_pred=grid_DT_pca.predict(X_train)
print("Mean Squared Error: ",metrics.mean_squared_error(y_train,y_pred))

Mean Squared Error:  696.5691486936969


It looks like the mean squared error is medium,not so hight not so low.

We have completed feature selection using PCA where we used grid searchcv , R-squared and mean squared error to check the accuracy.Now we are going to use selectkbest to get the optimum features for  the feature selection and will use the same methods to validate the accuracy of the model

In [240]:
n_indices = 8
X_train.columns[indices[:n_indices]]

Index(['Wheelbase', 'Length', 'Price_in_thousands', 'Year_Resale_Value',
       'Width', 'Power_perf_factor', 'Horsepower', 'Fuel_capacity'],
      dtype='object')

In [242]:
#selectkbest to get the scores of the best parameters
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=1)
names = X_train.columns[indices[:n_indices]]
skb_test = SelectKBest(score_func=f_regression,k=n_indices)
X_train_kbest = skb_test.fit_transform(X_train, y_train)
for n in range(0, len(names)):
 print(names[n], skb_test.scores_[n])


Wheelbase 8.438299882694018
Length 9.442760665349974
Price_in_thousands 0.032121195318380376
Year_Resale_Value 3.8193782923248536
Width 20.339651897458115
Power_perf_factor 3.531395140114711
Horsepower 10.998690940404886
Fuel_capacity 0.001409436087695959


C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Using selectkbest , we have got 8 best features. Now we will use these methods  to validate the accuracy of the model


1.GridsearchCV

2.R-squared

3.Mean Square Error



In [243]:
#Gridsearchcv
n_indices = 8
X_train.columns[indices[:n_indices]]
skb = SelectKBest(k=n_indices)
skb.fit(X_train, y_train)
X_train_skb = skb.transform(X_train)
X_test_skb = skb.transform(X_test)

C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Usually its a good practice to assign max depth as highest as possible.Here we have chose it as 50.Max_features is 11.Now,we will gridsearchcv to get the best set of parameters 

In [244]:
#Using Grid search  for finding the best set of parameters with random Forest model

dt_grid_skb={'max_depth':[50],'max_features': [11]}
grid_DT_skb=model_selection.GridSearchCV(rfc,dt_grid,cv=6)
grid_DT_skb.fit(X_train, y_train)

#grid_class_pca=model_selection.GridSearchCV(random_f_pca,dt_grid_pca,refit=True,return_train_score=True)
#grid_class_pca.fit(X_train_pca, y_train_pca)
#Displaying the parameters using Grid search
results_skb = grid_DT_pca.cv_results_
print(results_skb.get('params'))
print(results_skb.get('mean_test_score'))
print(results_skb.get('mean_train_score'))
print(grid_DT_skb.best_params_)
print(grid_DT_skb.best_score_)
final_model_pca = grid_DT_pca.best_estimator_
print(final_model_pca)

C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anacond

[{'max_depth': 10, 'max_features': 8}]
[-0.03784745]
[0.85985436]
{'max_depth': 5, 'max_features': 11}
0.009726630777253928
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features=8, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)


C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


In [245]:
# Calculate R-squared to get the accuracy
print("R-squared :" ,grid_DT_skb.score(X_train,y_train))



R-squared : 0.7346831295700225


In [ ]:
The R-squared  accuracy is little low.so looks like PCA is better to get the accuracy of the model

In [246]:
#Validating model accuracy using mean square error

y_pred_skb=grid_DT_skb.predict(X_train)
print("Mean Squared Error: ",metrics.mean_squared_error(y_train,y_pred_skb))

Mean Squared Error:  1397.8124638720835


In [ ]:
The mean squared error here for Seleckbest feature selection came to be on the higher side.

In [247]:
#Total time took to run the entire model
print(datetime.now()-start)


0:04:04.953249
